In [3]:
import tensorflow as tf
import numpy as np
import sys
import os
from six.moves import cPickle as pickle
import functools
import import_ipynb

In [4]:
img_size = 28
num_channels = 1
num_classes = 10
learning_rate = 1e-4
#ADD MORE PARAMETERS HERE BOI
params = {"learning_rate": learning_rate,
          "img_size": img_size,
          "num_channels":num_channels,
          "num_classes":num_classes}

#### Following code serves for nice and smart defyning model as reusable code

In [11]:
def lazy_property(function):
    attribute = '_cache_' + function.__name__
    
    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            with tf.variable_scope(function.__name__):
                setattr(self, attribute, function(self))
        return getattr(self, attribute)
    
    return decorator

##### Data loading Block, change that block from .ipynb to .py

In [12]:
from my_load_data import load_dataset_fn
x,y = load_dataset_fn('train')
print(x.shape, y.shape)

(200000, 28, 28, 1) (200000, 10)


##### Defining model as reusable function

In [39]:
class Model:
    
    def __init__(self, data, labels, params, mode=None):
        self.data = data
        self.target = labels
        self.img_size = params["img_size"]
        self.num_channels = params["num_channels"]
        self.num_classes = params["num_classes"]
        self.mode = mode
        self.prediction
        self.optimize
        self.error
        
    @lazy_property
    def prediction(self):
        #INPUT LAYER
        input_layer = tf.reshape(self.data,[-1, self.img_size, self.img_size, self.num_channels])
        #1 conv layer
        conv1 = tf.layers.conv2d(inputs = input_layer, 
                                 filters = 16,
                                 kernel_size = 5,
                                 strides = 1,
                                 padding = "same",
                                 activation = tf.nn.relu)
        #1 pool layer, img size reduced by 1/4
        pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                        pool_size = 2, 
                                        strides = 2,
                                        padding = "same")


        pool_1_flat = tf.reshape(pool1,[-1,int(self.img_size * self.img_size * 16 / 4)])


        dense = tf.layers.dense(inputs = pool_1_flat,
                                units = 128,
                                activation = tf.nn.relu)

        dense2 = tf.layers.dense(inputs = dense,
                                 units = 64,
                                 activation = tf.nn.relu)

        logits = tf.layers.dense(inputs = dense2,
                                 units = self.num_classes,
                                 activation = tf.nn.relu)




        # Softmax output of the neural network.
        #[10,x] tesnor
        y_pred = tf.nn.softmax(logits=logits)
        y_pred_cls = tf.argmax(y_pred, axis=1)
        return y_pred
    
    
    
    @lazy_property
    def optimize(self):
        cross_entropy =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.target, logits=predcition)
        optimizer = tf.train.AdamOptimizer(learning_rate = params["learning_rate"])
        return optimizer.minimize(cross_entropy)

    @lazy_property
    def error(self):
        mistakes = tf.not_equal(
            tf.argmax(self.label, 1), tf.argmax(self.prediction, 1))
        return tf.reduce_mean(tf.cast(mistakes, tf.float32))
    
    
    

    

In [ ]:
def main(dataset, labels, params, mode = None):
    sess = tf.Session()
    model = Model(dataset, labels, params, mode = None)
    sess.run(tf.initialize_all_variables())
    num_steps = 1001
    with tf.Session() as session:
        tf.global_variables_initializer().run()
        print('Initialized')
        for step in range(num_steps):
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
                [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 100 == 0):
                print('Minibatch loss at step %d: %f' % (step, l))
                print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
                print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
        print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

    
if __name__ == '__main__':
    main(x,y,params)

###### trash code
img_size_flat = int(img_size * img_size * 64/16)